### 14.5 사전학습된 BERT 모형의 직접 사용방법

#### [허깅페이스](https://huggingface.co/transformers)
- 자연어 분야에서 가장 널리 알려진 AI 커뮤니티.
- 트랜스포머, BERT, GPT 등 거의 모든 자연어 처리 딥러닝 모형들을 제공

```python
from transformers import pipeline
pipeline('sentiment-analysis')   # 감성분석
pipeline('text-classification')  # 문서분류
pipeline('question-answering')   # 질의응답
pipeline('text-generation')      # 문서생성
pipeline('translation')          # 기계번역
pipeline('summarization')        # 문서요약
```

- BERT는 기본적으로 많은 양의 메모리를 사용한다.
- pipeline에서는 자동으로 토크나이저를 선택해 사용할 때 기본 옵션을 사용하는데 , 이 경우 토큰 수가 많아지면 토크나이저는 문제가 없지만 이후 모형을 돌릴 때 에러가 나는 경우가 많다. 따라서 긴 문서의 경우에는 pipeline을 사용하지 않고 직접 토크나이저와 모형을 사용해야 한다.

In [1]:
from transformers import pipeline

clf = pipeline("sentiment-analysis")  # 감성분석
result = clf("what a beauiful day!")[0]
print("감성분석 결과 : {}, 감성스코어 : {:.4f}".format(result['label'], result['score']))

d:\python_project\파이썬 데이터분석 폴더\python_textmining\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
d:\python_project\파이썬 데이터분석 폴더\python_textmining\venv\lib\site-packages\torch\cuda\__init__.py:107: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of th

감성분석 결과 : NEGATIVE, 감성스코어 : 0.9822


In [2]:
clf("what a beauiful day!")

[{'label': 'NEGATIVE', 'score': 0.982161819934845}]

In [3]:
from transformers import pipeline

text_generator = pipeline("text-generation")  # 문장생성, GPT-2 사용
result = text_generator("Alice was beginning to get very tired of sitting by her sister on the bank, ")
print(result[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
d:\python_project\파이썬 데이터분석 폴더\python_textmining\venv\lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Alice was beginning to get very tired of sitting by her sister on the bank,  being overwhelmed by her own problems. In her mind, she was the next person to get caught, and was about to start making excuses for her sister when she


### 자동 클래스를 이용한 토크나이저와 모형의 사용

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Auto Classes를 이용해 사전학습된 내용에 맞는 토크나이저와 모형을 자동으로 설정
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased-finetuned-mrpc"
)

# 의미적으로 유사한 두 문장을 선언
input_sentence = "She angered me with her inappropriate comments, rumor-spreading, and disrespectfulness at the formal dinner table"
target_sentence = "She made me angry when she was rude at dinner"

# 토큰화
tokens = tokenizer(input_sentence, target_sentence, return_tensors="pt")

# 모형으로 결과를 예측
logits = model(**tokens).logits  # softmax 적용하기 전의 신경망 결과

# 소프트맥스를 이용해 결괏값을 클래스에 대한 확률로 변환
results = torch.softmax(logits, dim=1).tolist()[0]

In [5]:
for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

no: 29%
yes: 71%


In [6]:
target_sentence = "The boy quickly ran across the finish line, seizing yet another victory"

tokens = tokenizer(input_sentence, target_sentence, return_tensors="pt")
logits = model(**tokens).logits

results = torch.softmax(logits, dim=1).tolist()[0]

for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

no: 95%
yes: 5%


In [7]:
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np

nltk.download('movie_reviews')

# movie review data에서 file id를 가져옴
fileids = movie_reviews.fileids()

# file id를 이용해 raw text file을 가져옴
reviews = [ movie_reviews.raw(fileid) for fileid in fileids ]
categories = [ movie_reviews.categories(fileid)[0] for fileid in fileids ]

# label을 0. 1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = np.array([ label_dict[c] for c in categories ])

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)

print(f'Train set count : {len(X_train)}')
print(f"Test set count : {len(X_test)}")

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\namyo\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Train set count : 1600
Test set count : 400


In [8]:
# pipeline을 사용하면 에러 발생 -> BERT를 사용
# 테스트셋이 여러 개로 이뤄져서 배치로 처리
# 많은 메모리를 사용하므로 400개의 테스트셋을 10개씩 잘라서 모형을 돌리고 결과를 합쳐서 성능을 살펴봄

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# cuda를 이용한 GPU연산이 가능하면 cuda를 사용하고, 아니면 cpu를 사용
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Auto Classes를 이용해 사전학습된 내용에 맞는 토크나이저와 모형을 자동으로 설정
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"  # 미세조정학습이 된 모형
)

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

# 모델을 gpu로 옮겨서 연산을 준비
model = model.to(device)

batch_size = 10  # 모형으로 한 번에 예측할 데이터의 수
y_pred = []  # 전체 예측 결과를 저장

num_batch = len(y_test) // batch_size

for i in range(num_batch):
    inputs = tokenizer(
        X_test[i*batch_size: (i+1)*batch_size],  # X_test[0:10], X_test[10:20] ...
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    
    # 토큰화 결과를 GPU로 이동
    inputs = inputs.to(device)
    
    # 모형으로 결과를 예측
    logits = model(**inputs).logits
    
    # 결괏값을 클래스에 대한 확률로 변환
    pred = F.softmax(logits, dim=-1)
    
    # 예측결과를 CPU로 가져와서 넘파이로 변환한 후,
    # argmax로 확률이 가장 큰 클래스를 선택함.
    results = pred.cpu().detach().numpy().argmax(axis=1)
    
    # 전체 예측결과에 추가
    y_pred.extend(results.tolist())
    
# gpu 메모리를 비움
torch.cuda.empty_cache()

score = sum(y_test == np.array(y_pred)) / len(y_test)
print(f"NLTK 영화리뷰 감성분석 정확도 : {score}")

NLTK 영화리뷰 감성분석 정확도 : 0.8425
